# 目的
spaCyのdisplaCyを用いて、文章とタグを可視化する

In [1]:
import polars as pl

import spacy
from spacy.tokens import Span
from spacy import displacy

In [2]:
# 一度だけ必要
#!poetry run python -m spacy download en_core_web_sm

In [3]:
# https://matplotlib.org/2.0.2/examples/color/named_colors.html
options = {
    "colors": {
        "B-NAME_STUDENT": "aqua",
        "I-NAME_STUDENT": "aliceblue",
        "B-EMAIL": "forestgreen",
        "I-EMAIL": "greenyellow",
        "B-USERNAME": "hotpink",
        "I-USERNAME": "lightpink",
        "B-ID_NUM": "purple",
        "I-ID_NUM": "rebeccapurple",
        "B-PHONE_NUM": "red",
        "I-PHONE_NUM": "salmon",
        "B-URL_PERSONAL": "slategrey",
        "I-URL_PERSONAL": "silver",
        "B-STREET_ADDRESS": "brown",
        "I-STREET_ADDRESS": "chocolate",
    }
}

In [4]:
train = pl.read_json("../data/train.json")

In [5]:
documents = train.get_column("document").to_list()
full_texts = train.get_column("full_text").to_list()
labels = train.get_column("labels").to_list()

In [6]:
class PllLabelRenderer:
    def __init__(
        self,
        documents: list[str],
        full_texts: list[str],
        labels: list[str],
        options: dict[str, dict[str, str]],
    ) -> None:
        self.nlp = spacy.blank("en")
        self.documents = documents
        self.full_texts = full_texts
        self.labels = labels
        self.options = options

    def render(
        self, render_idx: int | None = None, documents_id: int | None = None
    ) -> None:
        if render_idx and documents_id:
            raise Exception("Don't Use Both render_idx and documents_id")
        if documents_id:
            render_idx = self.documents.index(documents_id)
        doc = self.nlp(self.full_texts[render_idx])
        doc.ents = [
            Span(doc, idx, idx + 1, label=label)
            for idx, label in enumerate(self.labels[render_idx])
            if label != "O"
        ]
        displacy.render(doc, style="ent", jupyter=True, options=options)

In [7]:
PLR = PllLabelRenderer(
    documents=documents, full_texts=full_texts, labels=labels, options=options
)

In [8]:
PLR.render(documents_id=104)

In [8]:
PLR.render(documents_id=9854)

In [45]:
PLR.render(render_idx=11)